In [1]:
%pylab qt
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Load the ggems files

In [2]:
# Loop through the ggems mhd files and save them to an array
import glob
import fastcat as fc
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
from mpl_interactions import ipyplot as iplt
# Zoom the ggems projections
from scipy.ndimage import zoom

ggems_scatter_files = glob.glob(
    '/home/jericho/1-Workspace/1-jupyter_notebooks/4-preliminary_fastmc_projections/out_fastmc/fastmc_*scatter.mhd'
)
ggems_primary_files = glob.glob(
    '/home/jericho/1-Workspace/1-jupyter_notebooks/4-preliminary_fastmc_projections/out_fastmc/fastmc_*[!scatter].mhd'
)

ggems_scatter_files.sort()
ggems_primary_files.sort()

ggems_primary_projections = []
ggems_scatter_projections = []

for ggems_scatter_file, ggems_primary_file in zip(ggems_scatter_files, ggems_primary_files):
    ggems_prmiary, b, c = fc.utils.read_mhd(ggems_primary_file)
    ggems_scatter, b, c = fc.utils.read_mhd(ggems_scatter_file)

    ggems_primary_projections.append(ggems_prmiary.squeeze())
    ggems_scatter_projections.append(ggems_scatter.squeeze())

# Load the flood file
flood_file = '/home/jericho/1-Workspace/1-jupyter_notebooks/4-preliminary_fastmc_projections/out_fastmc/flood_fastmc.mhd'
ggems_flood, b, c = fc.utils.read_mhd(flood_file)
ggems_flood = ggems_flood.squeeze()

# Load the pickled phantom
pkl_file = '/home/jericho/1-Workspace/1-jupyter_notebooks/2-fastcat_ggems/test/ggems_1e09_121kVp.pkl'
phantom = np.load(pkl_file, allow_pickle=True)

# Convert the phantom angles to degrees
phantom_angles = phantom.angles * 180 / np.pi

# Define the denoising functions
def downsize_block(image, block_size):
    image_size = image.shape[0]
    block_size = int(block_size)
    downsize = image_size//block_size
    image = image[:downsize*block_size, :downsize*block_size]
    image = image.reshape(downsize, block_size, downsize,
                          block_size).mean(axis=(1, 3))
    return image

def denoise_projections(projections):
    projections_zoomed = []

    projections = np.array(projections)
    # Check if the array is 3D
    if projections.ndim == 2:
        projections = projections[np.newaxis, :, :]
    for projection in projections:
        projection_zoomed_downsized = downsize_block(projection, 64)
        projection_zoomed = zoom(
            projection_zoomed_downsized, 64, order=4, mode='nearest')
        # Adjust the mean of the zoomed image to match the original
        projection_zoomed = projection_zoomed * \
            np.mean(projection) / np.mean(projection_zoomed)
        projections_zoomed.append(projection_zoomed)
    
    return projections_zoomed

# zoom the scatter projections using the zoom fromfunction
ggems_scatter_denoised = denoise_projections(np.array(ggems_scatter_projections).sum(axis=0))

Fastcat simulation

In [3]:
# Get the simulation parameters
simulation_parameters = phantom.simulation_parameters
detector_material = simulation_parameters['detector_material']
spectrum_file = '/home/jericho/1-Workspace/opengate_ggems_comparison/water_cylinder/data/spectrum_120kVp_2mmAl.dat'

# Load the spectrum as a fastcat spectrum object
s = fc.Spectrum()
s.load(spectrum_file=spectrum_file)

det = fc.Detector(s,'CsI-784-micrometer')
angles = phantom.angles
phantom.bowtie_file = '/home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat'

# return the projections
phantom.return_projs(det,s,angles,mgy=0,
                    ASG=False, 
                    scat_on=False,
                    det_on=False,
                    bowtie=True,
                    return_intensity=True)

[2024-01-24 14:30:15,467] {ggems_simulate.py:227} INFO - Initializing filter /home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat


AttributeError: 'Phantom' object has no attribute 'bowtie_interp'

Compare the two

In [142]:
# Define a standard scaler function
def standard_scaler(image):
    image = image - np.mean(image)
    image = image / np.std(image)
    return image

gg_primary = ggems_primary_projections[0] - ggems_scatter_denoised[0]
# Scale the first
plt.figure()
plt.subplot(1,2,1)
plt.imshow(standard_scaler(np.rot90(phantom.intensity[0],3)),
           vmin=np.percentile(standard_scaler(np.rot90(phantom.intensity[0],3)),1),
           vmax=np.percentile(standard_scaler(np.rot90(phantom.intensity[0],3)),99))
plt.title('Primary')
plt.subplot(1,2,2)
# Plot the ggems primary
plt.imshow(standard_scaler(gg_primary),
           vmin=np.percentile(standard_scaler(gg_primary),1),
           vmax=np.percentile(standard_scaler(gg_primary),99))

plt.figure()
pli.heatmap_slicer(np.arange(gg_primary.shape[0]), np.arange(gg_primary.shape[1]),
                   [standard_scaler(gg_primary), 
                    standard_scaler(np.rot90(phantom.intensity[0],3))],slices='both')


(<Figure size 1800x900 with 4 Axes>,
 array([<Axes: title={'center': 'heatmap_0'}, xlabel='X', ylabel='Y'>,
        <Axes: title={'center': 'heatmap_1'}, xlabel='X'>,
        <Axes: title={'center': 'Horizontal'}>,
        <Axes: title={'center': 'Vertical'}>], dtype=object))

In [147]:
plt.figure()
plt.plot(phantom.bowtie_coef * phantom.w_fluence_times_p_detected)
# plt.plot(phantom.w_fluence_times_p_detected)
plt.legend(phantom.energies)

In [153]:
plt.figure()
plt.imshow(phantom.geometric_falloff)

In [5]:
# Make a flat field taking into account the bowtie filter


# Get the bowtie filter
bowtie_filter = simulation_parameters['kwargs']['bowtie_file']
print(bowtie_filter)

NameError: name 'simulation_parameters' is not defined

The rest is stuff to build the functions that are now in ggems_simulate

In [6]:
# Based on the geometry find the angle of each pixel from the source
# Define the source position (z)
source_position = phantom.geomet.DSD
# Define the pixel position in the detector plane
pixel_position = np.linspace(-phantom.geomet.sDetector[0]/2, phantom.geomet.sDetector[0]/2, phantom.geomet.nDetector[0])
# Calculate the angle of each pixel from the source
pixel_angles = np.rad2deg(np.arctan(pixel_position/source_position))
# Plot the pixel angles
# plt.figure()
# plt.plot(pixel_angles)
# plt.title('Pixel angles')
# plt.xlabel('Pixel number')
# For each pixel angle find the bowtie filter thickness
# Load the bowtie filter
bowtie_filter = '/home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat'
# Define a function to parse the bowtie filter
def parse_bowtie_filter(bowtie_file):
    # read the bowtie file
    with open(bowtie_file, 'r') as f:
        bowtie_filter = f.read()
    bowtie_filter_columns = {}
    # Split the bowtie filter into lines
    bowtie_filter_lines = bowtie_filter.split('\n')
    bowtie_filter_columns['material'] = bowtie_filter_lines[2]
    bowtie_filter_columns['density'] = float(bowtie_filter_lines[4])
    bowtie_filter_columns['angular_increment'] = float(bowtie_filter_lines[6])
    bowtie_filter_columns['start_angle'] = float(bowtie_filter_lines[8])
    bowtie_filter_columns['thickness'] = np.array(bowtie_filter_lines[10:-1]).astype(float)
    bowtie_filter_columns['angles'] = np.arange(bowtie_filter_columns['start_angle'], bowtie_filter_columns['start_angle'] + bowtie_filter_columns['angular_increment'] * len(bowtie_filter_columns['thickness']), bowtie_filter_columns['angular_increment'])
    return bowtie_filter_columns

bowtie_dict = parse_bowtie_filter(bowtie_filter)


In [7]:
# Using the bowtie filter thicknesses and the pixel angles calculate the thickness of the bowtie filter at each pixel
# Define a function to calculate the bowtie filter thickness at each pixel
def calculate_bowtie_thicknesses(bowtie_dict, pixel_angles):
    # Interpolate the bowtie filter thicknesses to the pixel angles using nearest neighbour interpolation
    bowtie_thicknesses = np.interp(pixel_angles, bowtie_dict['angles'], bowtie_dict['thickness'], left=0, right=0)
    return bowtie_thicknesses

plt.figure()
plt.plot(pixel_angles, calculate_bowtie_thicknesses(bowtie_dict, pixel_angles))

bowtie_by_angle = calculate_bowtie_thicknesses(bowtie_dict, pixel_angles)

# Make an array that is the radial distance from the center of the image
def radial_distance(image,phantom):
    pixel_size = phantom.geomet.dDetector[0]
    x, y = np.meshgrid(np.arange(image.shape[0]), np.arange(image.shape[1]))
    x = x - image.shape[0] / 2 - phantom.geomet.offDetector[0]
    y = y - image.shape[1] / 2 - phantom.geomet.offDetector[1]
    x = x * pixel_size
    y = y * pixel_size
    return np.sqrt(x**2 + y**2)

# Make an array measuring the intensity in the projection as a function of radial distance
def r_squared_falloff(image,phantom):
    r = radial_distance(image,phantom)
    return (phantom.geomet.DSD / (r**2 + phantom.geomet.DSD**2)**0.5)**3

# Compare the last twenty columns of the images in a plot
proj_only = np.array(ggems_primary_projections) - np.array(ggems_scatter_projections)

plt.figure()
# plt.plot(ggems_primary_projections[23][:-40,-20:].mean(axis=1))


r_sq = r_squared_falloff(ggems_primary_projections[0], phantom)
plt.imshow(r_sq)

In [8]:
plt.figure()
plt.imshow(ggems_flood)

In [ ]:
# Plot the sum on the x and y axes of the flood image
plt.figure()
plt.plot(ggems_flood.sum(axis=0))
plt.plot(ggems_flood.sum(axis=1))


In [35]:
# Using scipy optimize find the best fit parameters for the flood image
from scipy.optimize import minimize

def fit_func(a):
    return np.mean(np.abs(ggems_flood.T - ( 10000 * a[0] * r_sq)*(np.exp(-a[1] * bowtie_by_angle))))

# Define the initial guess
a = 1
c = 1

# Define the bounds
a_bounds = (0, 10)
c_bounds = (0, 1)

# Do the minimization
res = minimize(fit_func, x0=(a, c), bounds=(a_bounds, c_bounds))


In [36]:
res

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1244.9483672103413
        x: [ 3.752e+00  6.141e-02]
      nit: 10
      jac: [-3.966e+02  2.423e+04]
     nfev: 99
     njev: 33
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

In [5]:
ggems_primary_only = np.array(ggems_primary_projections) - np.array(ggems_scatter_projections)

# Set the mean and standard deviation of the ggems primary only projections to match the fastcat projections
def normalize_mean_and_std(array1,array2):
    array1 = array1 - np.mean(array1)
    array1 = array1 * np.std(array2) / np.std(array1)
    array1 = array1 + np.mean(array2)
    return array1

phantom_projections = normalize_mean_and_std(phantom.intensity, ggems_primary_only)

In [5]:

# Set up a shallow neural network to map the fastcat projections to the ggems_primary_only projections using sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    phantom.intensity[0].flatten()[::4], ggems_primary_only[0].flatten()[::4], test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
scaler.fit(X_train.reshape(-1, 1))
X_train_scaled = scaler.transform(X_train.reshape(-1, 1))
X_test_scaled = scaler.transform(X_test.reshape(-1, 1))

# # Define the neural network
mlp = MLPRegressor(hidden_layer_sizes=(10,10,10),
                   activation='relu',
                   solver='adam',
                   max_iter=100,
                   verbose=True,
                   random_state=42)

# Use an svm instead of a neural network
# from sklearn.svm import SVR
# mlp = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)

# Train the neural network
mlp.fit(X_train_scaled, y_train)

# Predict the test data
predictions = mlp.predict(X_test_scaled)

# Plot the predictions vs the actual data
plt.figure()
plt.scatter(y_test, predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')

# Plot the residuals
plt.figure()
plt.scatter(y_test, y_test - predictions)
plt.xlabel('True Values')
plt.ylabel('Residuals')

# Calculate the mean squared error
print('Mean squared error: ', mean_squared_error(y_test, predictions))

Iteration 1, loss = 5079639.60294377
Iteration 2, loss = 4743419.72295433
Iteration 3, loss = 3097449.94515795
Iteration 4, loss = 667116.43853869
Iteration 5, loss = 8267.43574700
Iteration 6, loss = 3380.41746694
Iteration 7, loss = 3202.21059894
Iteration 8, loss = 3036.88847591
Iteration 9, loss = 2890.82199944
Iteration 10, loss = 2775.14262832
Iteration 11, loss = 2686.47855669
Iteration 12, loss = 2624.60414570
Iteration 13, loss = 2592.25198908
Iteration 14, loss = 2562.18994358
Iteration 15, loss = 2531.70409891
Iteration 16, loss = 2504.00823533
Iteration 17, loss = 2474.98991385
Iteration 18, loss = 2446.59877426
Iteration 19, loss = 2419.89894334
Iteration 20, loss = 2390.46073799
Iteration 21, loss = 2363.05582476
Iteration 22, loss = 2335.38503039
Iteration 23, loss = 2311.40902066
Iteration 24, loss = 2292.19990847
Iteration 25, loss = 2274.52774649
Iteration 26, loss = 2260.67834839
Iteration 27, loss = 2252.57437219
Iteration 28, loss = 2242.39554051
Iteration 29, loss

/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/u

In [7]:
# Use the neural network to predict the ggems primary only projections from the fastcat projections
ggems_primary_only_nn = []
for projection in phantom.intensity:
    projection_scaled = scaler.transform(projection.flatten().reshape(-1, 1))
    projection_nn = mlp.predict(projection_scaled)
    projection_nn = projection_nn.reshape(projection.shape)
    ggems_primary_only_nn.append(projection_nn)

# scale the ggems neural network projections to match the fastcat projections
ggems_fastcat_nn = normalize_mean_and_std(np.array(ggems_primary_only_nn), np.array(ggems_primary_only))

In [8]:
perc_diff_primary = (np.array(ggems_primary_only)-np.array(ggems_fastcat_nn[-1::-1]))/np.max(ggems_primary_only)*100

plt.figure()
ctrl = pli.hyperslicer(np.array(perc_diff_primary), cmap='bwr',
                        vmax=5,vmin=-5)

In [23]:
phantom_projections = ggems_fastcat_nn

# Downsize the perc_diff using the downsize_block function
perc_diff = (np.array(ggems_scatter_denoised)-np.array(ggems_scatter_projections))/np.max(ggems_scatter_projections)*100
perc_diff_small = []
for i in range(len(perc_diff)):
    perc_diff_small.append(downsize_block(perc_diff[i], 16))

# Plot the ggems projections
fig, (axes) = plt.subplots(ncols=2, nrows=3, figsize=(5.8,10))

# Plot the denoised ggems projections
controls = pli.hyperslicer(np.array(ggems_scatter_denoised),
                           ax=axes[1][0], cmap='jet',vmax=np.percentile(np.array(ggems_scatter_denoised),99),
                           vmin=np.percentile(np.array(ggems_scatter_denoised),1),play_buttons=True)
axes[1][0].set_title('Denoised GGEMS scatter')

ggems_primary_only = np.array(ggems_primary_projections) - np.array(ggems_scatter_projections)

# Plot the ggems projections
c2 = pli.hyperslicer(np.array(ggems_primary_only), ax=axes[0][1], cmap='gray',controls=controls,
                     vmax=np.percentile(np.array(ggems_primary_only),99),
                     vmin=np.percentile(np.array(ggems_primary_only),1))
axes[0][1].set_title('GGEMS Primary')

# Plot the phantom phantom_projections
c1 = pli.hyperslicer(np.array(phantom_projections)[-1::-1], ax=axes[1][1], cmap='gray',controls=controls,
                     vmax=np.percentile(np.array(phantom_projections),99),
                     vmin=np.percentile(np.array(phantom_projections),1))
axes[1][1].set_title('Analytical Primary')

# Plot the difference between the ggems and phantom projections
perc_diff_primary = (np.array(ggems_primary_only)-np.array(phantom_projections[-1::-1]))/np.max(phantom_projections)*100

c4 = pli.hyperslicer(np.array(perc_diff_primary), ax=axes[2][1], cmap='bwr',controls=controls,
                        vmax=5,vmin=-5)
axes[2][1].set_title('Perc. Diff. [%]')

# Add a colorbar
plt.colorbar(ax=axes[2][1])

# Plot the ggems projections
c2 = pli.hyperslicer(np.array(ggems_scatter_projections), ax=axes[0][0], cmap='jet',controls=controls,
                     vmax=np.percentile(np.array(ggems_scatter_projections),99),
                     vmin=np.percentile(np.array(ggems_scatter_projections),1))

axes[0][0].set_title('GGEMS Scatter')

# Plot the ggems primary projections
c3 = pli.hyperslicer(np.array(perc_diff_small),ax=axes[2][0], cmap='bwr',vmax=5,vmin=-5,controls=controls)
plt.colorbar(ax=axes[2][0])
axes[2][0].set_title('Perc. Diff. [%]')
plt.tight_layout()

# Turn off the axes
for ax in axes.flatten():
    ax.axis('off')
# Define a function that given an angle and radius returns x and y coordinates
# def f(axis0,radius=220,offset=0,x0=256,y0=256,na=90):
#     angle = np.deg2rad(axis0*360/na)
#     x = radius * np.cos(angle+offset) + x0
#     y = radius * np.sin(angle+offset) + y0
#     return np.array([[x,y],[x,y],[x,y]])

# iplt.plot(f, ax=axes[4], a=(0.1,10), b=(0.1,10), parametric=True,ylim=[0,1],xlim=[1,na],play_buttons=True)
# plt.imshow(phantom.phantom[50, :, :], cmap='gray')
# iplt.scatter(f, ax=axes[3][0], controls=controls,play_buttons=True,c='r',marker='>',parametric=True,)

# Add a label in the image saying source position
# axes[3].text(50, 50, 'Source Position', color='r', fontsize=12)


[2023-12-11 13:14:51,452] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f78a62b7ed0>
[2023-12-11 13:14:51,880] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f78a6360750>


In [48]:
# Save the figure as a gif using mpl_interactions
# save as a gif
anim = controls.save_animation("phantoms_ggems_analytical+denoised.gif", fig, "axis0", interval=120)

[2023-12-07 16:51:50,649] {animation.py:1060} INFO - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
[2023-12-07 16:51:50,649] {animation.py:322} DEBUG - frame size in pixels is 580 x 964
[2023-12-07 16:51:50,650] {animation.py:338} INFO - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 580x964 -pix_fmt rgba -framerate 8.333333333333334 -i pipe: -filter_complex 'split [a][b];[a] palettegen [p];[b][p] paletteuse' -y phantoms_ggems_analytical+denoised.gif
[2023-12-07 16:51:50,655] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:50,748] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:50,836] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:50,927] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:51,015] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-07 16:51:51,105] {animati

In [24]:
# Plot the error between the ggems and phantom projections as a function of angle in terms of rmse 
rmse = []
rmse_scatter = []
for i in range(len(phantom_projections)):
    rmse_scatter.append(np.sqrt(np.mean(np.array(perc_diff_small[i])**2)))
    rmse.append(np.sqrt(np.mean(np.array(perc_diff_primary[i])**2)))

plt.figure()
plt.plot(angles,rmse_scatter,label='Scatter')
plt.plot(angles,rmse,label='Primary')
plt.xlabel('Angle [degrees]')
plt.ylabel('RMSE [%]')
plt.legend()
plt.title('RMSE vs Angle')


Text(0.5, 1.0, 'RMSE vs Angle')

In [102]:
mean_projection = np.mean(np.array(ggems_projections), axis=0)
mean_projection_zoom = zoom(downsize_block(
    mean_projection, 64), 64, order=5, mode='nearest')

pli.heatmap_slicer(np.arange(0, mean_projection.shape[0]),
                   np.arange(0, mean_projection.shape[1]),
                   (mean_projection, mean_projection_zoom),
                   cmap='jet',
                   slices='both')

(<Figure size 1800x900 with 4 Axes>,
 array([<Axes: title={'center': 'heatmap_0'}, xlabel='X', ylabel='Y'>,
        <Axes: title={'center': 'heatmap_1'}, xlabel='X'>,
        <Axes: title={'center': 'Horizontal'}>,
        <Axes: title={'center': 'Vertical'}>], dtype=object))

In [101]:
# Do the same thing but for RectBivariateSpline
from scipy.interpolate import RectBivariateSpline
import numpy as np

ggems_projections_downsized_spline = []
for ggems_projection in ggems_projections_downsized:
    # Initialize the spline
    spline = RectBivariateSpline(np.arange(ggems_projection.shape[0]), np.arange(
        ggems_projection.shape[1]), ggems_projection, kx=4, ky=4, s=2)
    # Evaluate the spline on a finer grid
    x_fine, y_fine = np.meshgrid(np.linspace(0, ggems_projection.shape[0], ggems_projection.shape[0]*4), np.linspace(
        0, ggems_projection.shape[1], ggems_projection.shape[1]*4))
    z_fine = spline.ev(x_fine, y_fine)

    ggems_projections_downsized_spline.append(z_fine)

# Plot the ggems projections
plt.figure()
controls = pli.hyperslicer(
    np.array(ggems_projections_downsized_spline), cmap='jet')

Init signature:
RectBivariateSpline(
    x,
    y,
    z,
    bbox=[None, None, None, None],
    kx=3,
    ky=3,
    s=0,
)
Docstring:     
Bivariate spline approximation over a rectangular mesh.

Can be used for both smoothing and interpolating data.

Parameters
----------
x,y : array_like
    1-D arrays of coordinates in strictly ascending order.
    Evaluated points outside the data range will be extrapolated.
z : array_like
    2-D array of data with shape (x.size,y.size).
bbox : array_like, optional
    Sequence of length 4 specifying the boundary of the rectangular
    approximation domain, which means the start and end spline knots of
    each dimension are set by these values. By default,
    ``bbox=[min(x), max(x), min(y), max(y)]``.
kx, ky : ints, optional
    Degrees of the bivariate spline. Default is 3.
s : float, optional
    Positive smoothing factor defined for estimation condition:
    ``sum((z[i]-f(x[i], y[i]))**2, axis=0) <= s`` where f is a spline
    function. Defa

In [81]:
phantom = np.load(
    '/home/jericho/Software/fastcat/fastcat/data/phantoms/ct_scan_head_mandible_large.npy')

In [84]:
plt.figure()
contr = pli.hyperslicer(phantom)

In [85]:
# load a binary file to an array
import numpy as np

phantom = np.load()

In [86]:
plt.figure()
pli.hyperslicer(phantom.phantom)

AttributeError: 'Controls' object has no attribute 'vbox'